In [ ]:
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision import transforms
from dataset import DogsDataset
#from torchvision.models import ResNet50_Weights
import matplotlib.pyplot as plt
from net import dogs_stanford
import torch

In [ ]:
#the source of this dataset is hugging face, by amaye15/stanford-dogs, #link: https://huggingface.co/datasets/amaye15/stanford-dogs 
df = load_dataset("amaye15/stanford-dogs") 
train_df = df['train'].to_pandas() 

In [ ]:
train_data_loader = DataLoader(train_df, batch_size=32, shuffle=True) 
model = dogs_stanford()
model.train() 

In [ ]:
#We use the correct format input for the model
train_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

dataset_train = DogsDataset(
    train_df,
    transform=train_transforms
)
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True,num_workers=4, pin_memory=True) 
#batch size of n, adjust according to your system's capacity

In [ ]:
optimizer = Adam(model.parameters(), lr=0.005)  
criterion = nn.CrossEntropyLoss()
numberepochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#for images, label in train_loader:
    #print(images.shape)
#if you use this line, the actual format of the torch tensor is batch size,channels, height,width, correct format for CNN

In [ ]:
loss_values = [] 

In [ ]:
for epoch in range(numberepochs):  # set number of epochs
    running_loss = 0.0

    for images, label in train_loader:
        images = images.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    loss_values.append(epoch_loss)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

In [ ]:
plt.plot(range(1, len(loss_values) + 1), loss_values)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.show()